In [1]:
import pandas as pd
import requests
from datetime import timedelta, date
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',200)

/Users/williamyee/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [11]:
# start = date(2017, 6, 1)
# end = date(2018, 6, 1)


# df = pd.DataFrame()

# for sdate in range((end - start).days):
#     fdate = (start + timedelta(sdate)).strftime('%y%m%d')
#     url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + fdate + ".txt"
#     try:
#         data = pd.read_csv(url)
#         data['file'] = fdate
#         df = df.append(data)
#     except:
#         pass

In [12]:
# df['date'] = pd.to_datetime(df.DATE)

In [13]:
# df['turnid'] = df['STATION'] + '/' + df['SCP'] + '/' + df['C/A']

In [14]:
# # Remove readings at unusual times (that don't land on an even hour)
# df = df[df.TIME.str.contains(':00:00', regex=False)]


# df = df.sort_values(['STATION','turnid','TIME'])


# df = df.rename(columns={ df.columns[10]: "EXITS" })

In [15]:
# pickle.dump(df, open('turnstiledata','wb'))

In [16]:
df = pickle.load(open('turnstiledata','rb'))

In [17]:
df['STATION']= df['STATION'] +'/' +df['LINENAME'] 

In [18]:
conditions = [  df.TIME.isin(['23:00:00','00:00:00','01:00:00','02:00:00']),
                df.TIME.isin(['03:00:00','04:00:00','05:00:00','06:00:00']),
                df.TIME.isin(['07:00:00','08:00:00','09:00:00','10:00:00']),
                df.TIME.isin(['11:00:00','12:00:00','13:00:00','14:00:00']),
                df.TIME.isin(['15:00:00','16:00:00','17:00:00','18:00:00']),
                df.TIME.isin(['19:00:00','20:00:00','21:00:00','22:00:00'])]

choices = ['00:00','04:00','08:00','12:00','16:00','20:00']

df['TIME'] = np.select(conditions, choices, default=df.TIME)

In [19]:
df = df.groupby(['STATION','turnid','date','TIME'], as_index=False).sum()

df['dayofweek'] = df.date.dt.day_name()



In [20]:
df['entries'] = df.ENTRIES.shift(-1) - df.ENTRIES
df['exits'] = df.EXITS.shift(-1) - df.EXITS
df['passages'] = df.entries + df.exits



In [21]:

# Delete last row for each turnstile, for which we can't calculate the net # entries
df = df.groupby(['turnid'], as_index=False).apply(lambda x: x.iloc[:-1])
#df['weekend/weekday'] = np.where(df['dayofweek'].isin( ['Saturday','Sunday']), 'Weekend','Weekday')

In [22]:
df = df.groupby(['STATION','date','dayofweek','TIME'], as_index=False).sum()

In [23]:
df['weekend/weekday'] = np.where(df['dayofweek'].isin( ['Saturday','Sunday']), 'Weekend','Weekday')

In [24]:
df_weekend= df[df['weekend/weekday']=='Weekend']
df_weekday= df[df['weekend/weekday']!='Weekend']

In [25]:
df_weekend_median = df_weekend.groupby(['STATION','dayofweek','TIME'], as_index=False).median()

df_weekend_count = df_weekend.groupby(['STATION','dayofweek','TIME'], as_index=False).count()
df_weekend_median['count'] = df_weekend_count.iloc[:,4]

df_weekend_median = df_weekend_median.drop(columns =['ENTRIES','EXITS'])

sorted_df_weekend=df_weekend_median[df_weekend_median['count']>4].sort_values(by='passages', ascending = False)

df_saturday_median = sorted_df_weekend[sorted_df_weekend['dayofweek']=='Saturday']
df_sunday_median = sorted_df_weekend[sorted_df_weekend['dayofweek']=='Sunday']


In [26]:
df_saturday_median

,STATION,dayofweek,TIME,entries,exits,passages,count
1036,34 ST-HERALD SQ/BDFMNQRW,Saturday,16:00,18724.0,18552.0,37189.0,52
1035,34 ST-HERALD SQ/BDFMNQRW,Saturday,12:00,14268.0,18416.5,32604.0,52
3679,GRD CNTRL-42 ST/4567S,Saturday,12:00,16249.5,15601.5,31895.0,52
3680,GRD CNTRL-42 ST/4567S,Saturday,16:00,16238.0,14840.5,31208.5,52
303,14 ST-UNION SQ/LNQR456W,Saturday,12:00,15268.5,15239.5,30606.0,52
304,14 ST-UNION SQ/LNQR456W,Saturday,16:00,16970.5,11623.0,28471.0,52
1084,34 ST-PENN STA/ACE,Saturday,16:00,12209.0,12449.0,24893.5,52
5169,TIMES SQ-42 ST/1237ACENQRSW,Saturday,16:00,11786.5,12973.5,24833.0,52
1083,34 ST-PENN STA/ACE,Saturday,12:00,11603.0,11697.0,23163.5,52
3678,GRD CNTRL-42 ST/4567S,Saturday,08:00,10498.0,12199.0,22751.5,52


In [27]:
df_sunday_median

,STATION,dayofweek,TIME,entries,exits,passages,count
3567,FULTON ST/2345ACJZ,Sunday,20:00,22628.5,23264.5,45861.0,52
1042,34 ST-HERALD SQ/BDFMNQRW,Sunday,16:00,15221.5,14749.0,30116.0,52
1041,34 ST-HERALD SQ/BDFMNQRW,Sunday,12:00,11164.0,15496.0,26532.5,52
3685,GRD CNTRL-42 ST/4567S,Sunday,12:00,12835.5,12623.0,25326.0,52
3686,GRD CNTRL-42 ST/4567S,Sunday,16:00,13356.5,11285.0,24853.5,52
309,14 ST-UNION SQ/LNQR456W,Sunday,12:00,11536.0,12403.0,23921.5,52
1090,34 ST-PENN STA/ACE,Sunday,16:00,12273.5,10510.0,22901.0,52
310,14 ST-UNION SQ/LNQR456W,Sunday,16:00,13186.0,8420.0,21736.5,52
1089,34 ST-PENN STA/ACE,Sunday,12:00,10348.0,10148.0,20661.5,52
5175,TIMES SQ-42 ST/1237ACENQRSW,Sunday,16:00,10070.0,9481.0,19395.5,52


In [28]:
df_weekday_median = df_weekday.groupby(['STATION','TIME'], as_index=False).median()

df_weekday_count = df_weekday.groupby(['STATION','TIME'], as_index=False).count()
df_weekday_median['count'] = df_weekday_count.iloc[:,4]

df_weekday_median = df_weekday_median.drop(columns =['ENTRIES','EXITS'])
df_weekday_median

,STATION,TIME,entries,exits,passages,count
0,1 AV/L,00:00,466.0,339.0,809.0,260
1,1 AV/L,04:00,1358.0,2919.0,4299.5,260
2,1 AV/L,08:00,4905.0,5845.0,10693.5,260
3,1 AV/L,12:00,4040.5,3950.0,8028.5,260
4,1 AV/L,16:00,6927.5,6392.5,13391.0,260
5,1 AV/L,20:00,4112.0,3737.0,7871.0,259
6,103 ST-CORONA/7,00:00,210.0,873.5,1097.0,260
7,103 ST-CORONA/7,04:00,7040.5,585.5,7609.5,260
8,103 ST-CORONA/7,08:00,5602.0,1391.0,6957.0,259
9,103 ST-CORONA/7,12:00,3024.0,2430.5,5523.0,260


In [29]:
sorted_df_weekday=df_weekday_median[df_weekday_median['count']>4].sort_values(by='passages', ascending = False)
sorted_df_weekday

,STATION,TIME,entries,exits,passages,count
1846,GRD CNTRL-42 ST/4567S,16:00,62367.0,34546.5,97212.0,260
520,34 ST-HERALD SQ/BDFMNQRW,16:00,42328.0,32793.5,75248.0,260
1844,GRD CNTRL-42 ST/4567S,08:00,28795.5,43813.0,72756.0,260
518,34 ST-HERALD SQ/BDFMNQRW,08:00,18355.5,37878.0,56319.5,260
1845,GRD CNTRL-42 ST/4567S,12:00,29329.0,24047.0,53580.5,260
544,34 ST-PENN STA/ACE,16:00,22799.0,27799.0,50922.0,260
1787,FULTON ST/2345ACJZ,20:00,24616.0,24358.0,49482.0,259
154,14 ST-UNION SQ/LNQR456W,16:00,31155.5,14381.0,45583.5,260
622,47-50 STS ROCK/BDFM,16:00,36858.0,6922.0,43636.5,260
542,34 ST-PENN STA/ACE,08:00,25275.0,17507.5,42691.5,260


In [30]:
rank_weekday=sorted_df_weekday.groupby('STATION', as_index = False).sum()
rank_weekday= rank_weekday.sort_values(by ='passages', ascending = False)
rank_weekday= rank_weekday['STATION'].reset_index().rename(columns={'index':'median index'})


In [31]:
top_weekday=rank_weekday[:10]

In [32]:
top_weekday

,median index,STATION
0,306,GRD CNTRL-42 ST/4567S
1,86,34 ST-HERALD SQ/BDFMNQRW
2,90,34 ST-PENN STA/ACE
3,25,14 ST-UNION SQ/LNQR456W
4,430,TIMES SQ-42 ST/1237ACENQRSW
5,99,42 ST-PORT AUTH/ACENQRS1237W
6,103,47-50 STS ROCK/BDFM
7,296,FULTON ST/2345ACJZ
8,119,59 ST COLUMBUS/ABCD1
9,283,FLUSHING-MAIN/7


In [33]:
rank_sat=df_saturday_median.groupby('STATION', as_index = False).sum()
rank_sat= rank_sat.sort_values(by ='passages', ascending = False)
rank_sat= rank_sat['STATION'].reset_index().rename(columns={'index':'median index'})



In [34]:

top_sat =rank_sat[:10]

In [35]:
rank_sun=df_sunday_median.groupby('STATION', as_index = False).sum()
rank_sun= rank_sun.sort_values(by ='passages', ascending = False)
rank_sun= rank_sun['STATION'].reset_index().rename(columns={'index':'median index'})



In [36]:

top_sun=rank_sun[:10]


In [37]:
tourist_index = df_weekend_median.groupby('STATION').sum()['passages']/df_weekday_median.groupby('STATION').sum()['passages']


In [38]:
top_weekday=top_weekday.merge(tourist_index, how='inner', left_on="STATION",right_index =True, suffixes= ('','_tourist_index'))


In [39]:
top_weekday=top_weekday.drop(['median index'],axis =1)
top_weekday = top_weekday.rename(columns={'passages':'touristy'})

In [40]:
top_weekday

,STATION,touristy
0,GRD CNTRL-42 ST/4567S,0.694771
1,34 ST-HERALD SQ/BDFMNQRW,0.993466
2,34 ST-PENN STA/ACE,1.053069
3,14 ST-UNION SQ/LNQR456W,1.205712
4,TIMES SQ-42 ST/1237ACENQRSW,1.156693
5,42 ST-PORT AUTH/ACENQRS1237W,1.278535
6,47-50 STS ROCK/BDFM,0.492084
7,FULTON ST/2345ACJZ,0.927825
8,59 ST COLUMBUS/ABCD1,1.123480
9,FLUSHING-MAIN/7,1.265176


In [41]:
top_sat

,median index,STATION
0,86,34 ST-HERALD SQ/BDFMNQRW
1,306,GRD CNTRL-42 ST/4567S
2,25,14 ST-UNION SQ/LNQR456W
3,99,42 ST-PORT AUTH/ACENQRS1237W
4,90,34 ST-PENN STA/ACE
5,428,TIMES SQ-42 ST/1237ACENQRSW
6,283,FLUSHING-MAIN/7
7,331,JKSN HT-ROOSVLT/EFMR7
8,119,59 ST COLUMBUS/ABCD1
9,231,CANAL ST/JNQRZ6W


In [42]:
top_sun

,median index,STATION
0,86,34 ST-HERALD SQ/BDFMNQRW
1,306,GRD CNTRL-42 ST/4567S
2,90,34 ST-PENN STA/ACE
3,25,14 ST-UNION SQ/LNQR456W
4,99,42 ST-PORT AUTH/ACENQRS1237W
5,296,FULTON ST/2345ACJZ
6,428,TIMES SQ-42 ST/1237ACENQRSW
7,283,FLUSHING-MAIN/7
8,331,JKSN HT-ROOSVLT/EFMR7
9,119,59 ST COLUMBUS/ABCD1


In [43]:
sorted_df_weekday

,STATION,TIME,entries,exits,passages,count
1846,GRD CNTRL-42 ST/4567S,16:00,62367.0,34546.5,97212.0,260
520,34 ST-HERALD SQ/BDFMNQRW,16:00,42328.0,32793.5,75248.0,260
1844,GRD CNTRL-42 ST/4567S,08:00,28795.5,43813.0,72756.0,260
518,34 ST-HERALD SQ/BDFMNQRW,08:00,18355.5,37878.0,56319.5,260
1845,GRD CNTRL-42 ST/4567S,12:00,29329.0,24047.0,53580.5,260
544,34 ST-PENN STA/ACE,16:00,22799.0,27799.0,50922.0,260
1787,FULTON ST/2345ACJZ,20:00,24616.0,24358.0,49482.0,259
154,14 ST-UNION SQ/LNQR456W,16:00,31155.5,14381.0,45583.5,260
622,47-50 STS ROCK/BDFM,16:00,36858.0,6922.0,43636.5,260
542,34 ST-PENN STA/ACE,08:00,25275.0,17507.5,42691.5,260


In [44]:
weekday_for_plot =df_weekday_median[df_weekday_median['STATION'].isin(top_weekday['STATION'])]

In [53]:
saturday_for_plot =df_weekday_median[df_weekday_median['STATION'].isin(top_weekday['STATION'])]

In [55]:
saturday_for_plot = saturday_for_plot.merge(tourist_index, left_on='STATION', right_index=True)

In [59]:
saturday_for_plot = saturday_for_plot.rename(columns={'passages_x':'passages', 'passages_y':'touristindex'})

In [61]:
saturday_for_plot['passages/tourist'] = saturday_for_plot['passages'] / saturday_for_plot['touristindex']


In [62]:
saturday_for_plot.sort_values()

,STATION,TIME,entries,exits,passages,count,touristindex,passages/tourist
150,14 ST-UNION SQ/LNQR456W,00:00,782.0,356.5,1161.5,260,1.205712,963.331478
151,14 ST-UNION SQ/LNQR456W,04:00,4099.5,13028.5,17184.5,260,1.205712,14252.578379
152,14 ST-UNION SQ/LNQR456W,08:00,9209.5,20153.0,29721.0,260,1.205712,24650.172074
153,14 ST-UNION SQ/LNQR456W,12:00,18597.5,13888.0,32745.5,260,1.205712,27158.649092
154,14 ST-UNION SQ/LNQR456W,16:00,31155.5,14381.0,45583.5,260,1.205712,37806.302572
155,14 ST-UNION SQ/LNQR456W,20:00,11096.0,3148.0,14211.0,259,1.205712,11786.400032
516,34 ST-HERALD SQ/BDFMNQRW,00:00,1150.0,725.5,1880.0,260,0.993466,1892.364826
517,34 ST-HERALD SQ/BDFMNQRW,04:00,5313.0,9723.5,15130.5,260,0.993466,15230.013828
518,34 ST-HERALD SQ/BDFMNQRW,08:00,18355.5,37878.0,56319.5,260,0.993466,56689.915321
519,34 ST-HERALD SQ/BDFMNQRW,12:00,17964.0,21966.5,39892.0,260,0.993466,40154.371079


In [50]:
df.columns

Index(['STATION', 'date', 'dayofweek', 'TIME', 'ENTRIES', 'EXITS', 'entries',
       'exits', 'passages', 'weekend/weekday'],
      dtype='object')

In [64]:
import seaborn as sns
import matplotlib.pyplot as plt

In [66]:
g = sns.FacetGrid(saturday_for_plot,col ='STATION', row = 1)
g = g.map(plt.hist('passages'))
g

KeyError: 1

In [71]:
plt.scatter(x=saturday_for_plot.passages, y=saturday_for_plot.TIME);

ValueError: could not convert string to float: '20:00'

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x10c0ecf28> (for post_execute):


ValueError: could not convert string to float: '20:00'

ValueError: could not convert string to float: '20:00'

In [72]:
tourist_index[]

STATION
1 AV/L                            1.411421
103 ST-CORONA/7                   1.354120
103 ST/1                          1.281425
103 ST/6                          1.059196
103 ST/BC                         1.116310
104 ST/A                          0.881225
104 ST/JZ                         0.846615
110 ST/6                          1.190481
111 ST/7                          1.434293
111 ST/A                          0.763056
111 ST/J                          0.859653
116 ST-COLUMBIA/1                 0.833397
116 ST/23                         1.247219
116 ST/6                          1.293345
116 ST/BC                         1.143987
121 ST/JZ                         0.845514
125 ST/1                          1.282812
125 ST/23                         1.210583
125 ST/456                        1.224339
125 ST/ACBD                       1.419974
135 ST/23                         1.163607
135 ST/BC                         0.855044
137 ST CITY COL/1                 1.146891
138